In [14]:
import tkinter as tk
from tkinter import ttk, messagebox
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import csv 
import os



In [15]:
# Loading the CSV files as dictionaries to respective variables 

with open('lecturers.csv', newline='', encoding='utf-8') as lecturers: 
    reader = csv.DictReader(lecturers)
    lecturers = list(reader)

with open('courses.csv', newline='', encoding='utf-8') as courses: 
    reader = csv.DictReader(courses)
    courses = list(reader)

with open('timeslots.csv', newline='', encoding='utf-8') as timeslots: 
    reader = csv.DictReader(timeslots)
    timeslots = list(reader)

with open('rooms.csv', newline='', encoding='utf-8') as rooms: 
    reader = csv.DictReader(rooms)
    rooms = list(reader)

In [16]:
def fitness_function(timetable, courses, rooms): 
    # 'timetable' is a list of 'session' tuples structured as (course_id, lecturer_id, day, timeslot, room_id)

    # Constant penalties & rewards for certain situations
    ROOM_OVERLOAD_PENALTY = 5
    CONFLICT_PENALTY = 10
    VALID_SESSION_REWARD = 1

    score, penalty = 0 

    room_time_slots = set()
    lecturer_time_slots = set()
    scheduled_courses = set()

    for session in timetable: 
        course_id, lecturer_id, day, timeslot, room_id = session 

        course = courses.get(course_id)
        room = rooms.get(room_id)

        if not course or not room: 
            penalty += CONFLICT_PENALTY
            continue 

        session_penalty = 0 

        # Room capacity constraint 
        if int(course["students_enrolled"]) > int(room["room_capacity"]): 
            session_penalty += ROOM_OVERLOAD_PENALTY
        
        # Room/time conflict constraint
        if (room_id, day, timeslot) in room_time_slots: 
            session_penalty += CONFLICT_PENALTY
        else: 
            room_time_slots.add((room_id, day, timeslot))

        # Lecturer/time conflict constraint 
        if (lecturer_id, day, timeslot) in lecturer_time_slots: 
            session_penalty += CONFLICT_PENALTY
        else: 
            lecturer_time_slots.add((lecturer_id, day, timeslot))

        if session_penalty == 0: 
            score += VALID_SESSION_REWARD
        else: 
            penalty += session_penalty

        # Higher fitness -> better
        fitness = score - penalty
        return fitness 


In [ ]:
course_ids = [c["course_id"] for c in courses]
lecturer_ids = [l["lecturer_id"] for l in lecturers]
room_ids = [r["room_id"] for r in rooms]
time_slots = [(t["day"], t["timeslot"]) for t in timeslots]

# Generates a random timetable with one session for each course 
def generate_timetable(): 
    timetable = []
    for course_id in course_ids: 
        lecturer_id = random.choice(lecturer_ids)
        room_id = random.choice(room_ids)
        day, timeslot = random.choice(time_slots)
        session = (course_id, lecturer_id, day, timeslot, room_id)
        timetable.append(session)
    return timetable 

[('1', '116', 'Sunday', '9:00-10:00', '206'), ('2', '101', 'Tuesday', '10:30-11:30', '208'), ('3', '118', 'Wednesday', '13:30-14:30', '207'), ('4', '119', 'Tuesday', '10:30-11:30', '206'), ('5', '103', 'Sunday', '13:30-14:30', '201'), ('6', '118', 'Wednesday', '9:00-10:00', '203'), ('7', '115', 'Tuesday', '9:00-10:00', '202'), ('8', '113', 'Thursday', '13:30-14:30', '207'), ('9', '118', 'Wednesday', '10:30-11:30', '207'), ('10', '102', 'Monday', '12:00-13:00', '211'), ('11', '116', 'Monday', '9:00-10:00', '208'), ('12', '120', 'Tuesday', '10:30-11:30', '205'), ('13', '118', 'Thursday', '13:30-14:30', '208'), ('14', '120', 'Monday', '12:00-13:00', '207'), ('15', '113', 'Wednesday', '10:30-11:30', '209'), ('16', '108', 'Thursday', '13:30-14:30', '212'), ('17', '105', 'Thursday', '15:00-16:00', '201'), ('18', '110', 'Thursday', '12:00-13:00', '201'), ('19', '108', 'Tuesday', '12:00-13:00', '210'), ('20', '105', 'Sunday', '12:00-13:00', '210')]


In [ ]:
# Uses the previous function to generate a list (size 'size') of timetables 
def initialise_population_rand(size): 
    return [generate_timetable() for _ in range(size)]

# Exponential rank based parent selection
def select_parents_rank_based(population, fitness_scores): 
    # Selects two parents based on the formula:
    # P(i) = (1-e^(-i)) / c, where i = 0 (best) to n-1 (worst)

    n = len(population)

    # Rank population by descending fitness
    sorted_indices = np.argsort(fitness_scores)[::-1] # i.e. highest fitness first
    sorted_population = [population[i] for i in sorted_indices]

    # Calculate raw probabilities 

    # creates a list  [0, 1, 2, 3, ..., n-1]
    ranks = np.arange(n) # 0 = best rank
    raw_probs = 1-np.exp(-ranks)

    # Normalises so sum of probabilities is 1 
    c = raw_probs.sum()
    probabilities = raw_probs / c

    # Selects two parents based on these probabilities
    selected_indices = np.random.choice(n, size=2, replace=False, p=probabilities)
    parent1 = sorted_population[selected_indices[0]]
    parent2 = sorted_population[selected_indices[1]]
    return parent1, parent2

def cycle_crossover(parent1, parent2): 
    # Produces a child using alternating cycles from parent1 and parent2

    size = len(parent1)
    child = [None] * size
    visited = [False] * size
    use_parent1 = True # to facilitate alternation between parents 

    while not all(visited): 
        # find first unvisted position
        start = next(i for i, v in enumerate(visited) if not v)
        idx = start
        cycle_indices = []

        while True: 
            cycle_indices.append(idx)
            visited[idx] = True
            element = parent2[idx]
            idx = parent1.index(element)
            if idx == start:
                break

            # assign genes
            for i in cycle_indices: 
                child[i] = parent1[i] if use_parent1 else parent2[i]

            # switch parent; changes each cycle
            use_parent1 = not use_parent1 

        return child 
    
def insert_mutate(individual): 
    # Performs insert mutation on an individual (list of sessions i.e. timetable)

    size = len(individual)
    if size < 2: 
        return individual.copy() # nothing to mutate
    
    mutated = individual.copy()

    # randomly pick two distinct positions
    i, j = random.sample(range(size), 2)

    # remove element at i 
    gene = mutated.pop(i)

    # insert it at j
    mutated.insert(j, gene)

    return mutated 

def survivor_selection_mu_lambda(offspring, fitness_scores, mu):
    # (μ, λ)-selection: pick top-μ from λ offspring

    # Sort offspring by descending fitness
    sorted_indices = sorted(range(len(offspring)), key=lambda i: fitness_scores[i], reverse=True)

    # Select top μ individuals
    next_generation = [offspring[i] for i in sorted_indices[:mu]]

    return next_generation 

# Diversity through shared fitness function
def course_order_distance(ind1, ind2):
    return sum(1 for a, b in zip(ind1, ind2) if a[0] != b[0])

def shared_fitness(population, raw_fitnesses, distance_func, sigma=1.0, alpha=1.0):
    """
    Parameters:
    - population: list of individuals (solutions)
    - raw_fitnesses: list of original fitness values f(i)
    - distance_func: function taking (ind1, ind2) and returning float distance
    - sigma: niche radius (σ)
    - alpha: sharing shape factor
    """

    mu = len(population)
    shared_fitnesses = []

    for i in range(mu): 
        sharing_sum = 0.0
        for j in range(mu): 
            if i == j: 
                d = 0
            else: 
                d = distance_func(population[i], population[j])
            
            # Sharing function sh(d) 
            if d <= sigma: 
                sh = 1 - (d / sigma) ** alpha
            else: 
                sh = 0

            sharing_sum += sh

            # avoid div by 0 
            if sharing_sum == 0:
                sharing_sum = 1e-8 
            
            f_shared = raw_fitnesses[i] / sharing_sum
            shared_fitnesses.append(f_shared)

    return shared_fitnesses

In [ ]:
# "Greedy" population initialisation functions. Operates within constraints.
# More likely to generate valid, or near-valid timetables that have higher fitness to begin with.
def initialise_population_greedy(size, courses, rooms):
    population = []

    for _ in range(size):
        timetable = []
        used_room_slots = set()
        used_lecturer_slots = set()

        for course_id in course_ids:
            assigned = False
        
            for day, timeslot in time_slots:
                for room_id in room_ids: 
                    for lecturer_id in lecturer_ids: 
                        room_slot = (room_id, day, timeslot)
                        lecturer_slot = (lecturer_id, day, timeslot)

                        # Room capacity constraint
                        if (int(courses["course_id"]["students_enrolled"]) > int(rooms["room_id"]["capacity"])): 
                            continue 

                        # Assign valid slot
                        session = (course_id, lecturer_id, day, timeslot, room_id)
                        timetable.append(session)
                        used_room_slots.add(room_slot)
                        used_lecturer_slots.add(lecturer_slot)
                        assigned = True 
                        break
                if assigned: break
            if assigned: break
        
        # If no valid slot is found, fall back to random initialisation (similar to the generate_timetable function)
        if not assigned: 
            lecturer_id = random.choice(lecturer_ids)
            room_id = random.choice(room_ids)
            day, timeslot = random.choice(time_slots)
            session = (course_id, lecturer_id, day, timeslot, room_id)
            timetable.append(session)

        population.append(timetable)

    return population 

# Fitness proportionate parent selection (FPS)
def parent_select_fitness_proportionate(population, fitness_scores): 
    
    n = len(population)
    # Convert fitness scores to probabilities
    total_fitness = sum(fitness_scores)

    # prevent div by 0 
    if total_fitness == 0: 
        probabilities = [1/len(population) * len(population)]
    else: 
        probabilities = [f / total_fitness for f in fitness_scores]

    # Select two parents based on probabilities 
    selected_indices = np.random.choice(n, size=2, replace=False, p=probabilities)
    parent1 = population[selected_indices[0]]
    parent2 = population[selected_indices[1]]
    return parent1, parent2 

# Order crossover (returns two children)
def order_crossover(parent1, parent2):
    
    size = len(parent1)

    # pick two crossover points
    p1, p2 = sorted(random.sample(range(size), 2))

    # placeholder children
    child1 = [None] * size
    child2 = [None] * size

    # copy middle slice from each parent
    child1[p1:p2] = parent1[p1:p2]
    child2[p1:p2] = parent2[p1:p2]

    # fill the rest from other parent
    def fill_child(child, donor): 
        current_pos = p2
        donor_index = p2 % size

        while None in child: 
            gene = donor[donor_index % size]
            if gene not in child: 
                child[current_pos % size] = gene
                current_pos += 1
            donor_index += 1

    fill_child(child1, parent2)
    fill_child(child2, parent1)

    return child1, child2 

def scramble_mutate(individual): 
    
    mutated = individual.copy()
    size = len(mutated)

    if size < 2: 
        return mutated
    
    # select 2 distinct crossover points
    i, j = sorted(random.sample(range(size), 2))

    # scramble subsegment
    segment = mutated[i:j]
    random.shuffle(segment)
    mutated[i:j] = segment

    return mutated

def survivor_selection_round_robin(parents, offspring, fitness_func, mu, q=10):
    """
    Parameters:
    - parents: list of parent individuals (size μ)
    - offspring: list of offspring individuals (size μ)
    - fitness_func: function to evaluate fitness of an individual
    - mu: number of individuals to retain
    - q: number of opponents each individual faces (tournament size)
    """
    combined = parents + offspring 
    fitness_scores = [fitness_func(ind) for ind in combined]
    num_individuals = len(combined)
    win_counts = [0] * num_individuals

    for i in range(num_individuals): 
        opponents = random.sample([j for j in range(num_individuals) if j != i], k=min(q, num_individuals - 1))
        for j in opponents: 
            if fitness_scores[i] > fitness_scores[j]: 
                win_counts[i] += 1

    # Select top μ individuals with the most wins
    ranked_indices = sorted(range(num_individuals), key=lambda i: win_counts[i], reverse=True)
    selected_indices = ranked_indices[:mu]
    next_generation = [combined[i] for i in selected_indices]

    return next_generation

# def check_stagnation(best_fitness_history, window_size=10, tolerance=1e-6):
#     # Checks if fitness has stagnated over last 'window_size' generations.
#     # best_fitness_history: list of best fitness values per generation
#     # tolerance: minimum improvement required to avoid stagnation

#     if len(best_fitness_history) < window_size: 
#         return False
    
#     recent = best_fitness_history[-window_size:]
#     improvement = max(recent) - min(recent)
#     return improvement < tolerance

# # Replaces a % of the population with new individuals
# def apply_extinction(population, extinction_rate=0.5, regenerate_func=None): 
#     if regenerate_func is None: 
#         raise ValueError("Must provide function to regenerate population")
    
#     pop_size = len(population)
#     kill_num = int(extinction_rate * pop_size)

#     # Randomly choose survivors & generate replacements
#     survivors = random.sample(population, pop_size - kill_num)
#     new_individuals = regenerate_func(kill_num)

#     return survivors + new_individuals
    